# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Оглавление<a name="оглавление"></a>

1. [Знакомство с данными](#1.-Знакомство-с-данными)
2. [Предобработка данных](#2.-Предобработка-данных)
	1. [Обработка артефактов](#Обработка-артефактов)
	2. [Обработка пропусков](#Обработка-пропусков)
	3. [Замена типа данных](#Замена-типа-данных)
	4. [Обработка дубликатов](#Обработка-дубликатов)
	5. [Лемматизация](#Лемматизация)
	6. [Категоризация данных](#Категоризация-данных)
3. [Ответы на вопросы](#3.-Ответы-на-вопросы)
	1. [Есть ли зависимость между наличием детей и возвратом кредита в срок?](#Есть-ли-зависимость-между-наличием-детей-и-возвратом-кредита-в-срок?)
	2. [Есть ли зависимость между семейным положением и возвратом кредита в срок?](#Есть-ли-зависимость-между-семейным-положением-и-возвратом-кредита-в-срок?)
	3. [Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#Есть-ли-зависимость-между-уровнем-дохода-и-возвратом-кредита-в-срок?)
	4. [Как разные цели кредита влияют на его возврат в срок?](#Как-разные-цели-кредита-влияют-на-его-возврат-в-срок?)
4. [Общий вывод](#4.-Общий-вывод)

## Импорт библиотек

Перед началом работы импортируем библиотеки.

In [1]:
import pandas as pd
from pymystem3 import Mystem

In [2]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

## 1. Знакомство с данными

Сохраним датафрейм в переменную `df` и посмотрим на общую информацию.

In [3]:
df = pd.read_csv('data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Рассмотрим полученные данные.

В таблице 12 столбцов трех разных типов: `int64`, `float64` и `object`.

Вот какую информацию содержит каждый столбец:

* *children* — количество детей в семье
* *days_employed* — общий трудовой стаж в днях
* *dob_years* — возраст клиента в годах
* *education* — уровень образования клиента
* *education_id* — идентификатор уровня образования
* *family_status* — семейное положение
* *family_status_id* — идентификатор семейного положения
* *gender* — пол клиента
* *income_type* — тип занятости
* *debt* — имел ли задолженность по возврату кредитов
* *total_income* — ежемесячный доход
* *purpose* — цель получения кредита

Посмотрим на данные поближе.

In [4]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Сразу видны проблемы со столбцами `education` и `days_employed`. В столбце `education` значения записаны в разных регистрах:

In [5]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

В столбце `days_employed` есть отрицательные и положительные значения. Я заметил, что положительные значения очень большие: больше 300 тысяч рабочих дней — это почти век работы. При этом медианный возраст клиентов с положительным количеством рабочих дней — 60 лет. Я думаю, что положительные величинины — это не дни, а часы. С этим предположением получается, что медианное значение стажа — 34 года. Мне кажется, это правдоподобное значение.

Также я заметил, что клиенты с положительным стажем делятся на две группы: безработные и пенсионеры. 

In [6]:
print(color.BOLD + 'Количество клиентов с положительным стажем по типу дохода' + color.END)
print(df[df['days_employed'] > 0].groupby('income_type')['income_type'].count())

Количество клиентов с положительным стажем по типу дохода
income_type
безработный       2
пенсионер      3443
Name: income_type, dtype: int64


При этом безработных всего двое, и у них стаж больше, чем возраст.

In [7]:
unemployed = df[df['income_type']=='безработный'][['income_type', 'days_employed', 'dob_years']]
unemployed['days_employed'] = unemployed['days_employed'] / 24 / 365
unemployed.rename(columns={'days_employed': 'years_employed'}, inplace=True)
unemployed

,income_type,years_employed,dob_years
3133,безработный,38.530190,31
14798,безработный,45.125895,45


Поэтому думаю, что безработные клиенты — это выбросы в данных, поэтому от них надо будет избавиться.

Еще я решил проверить, какие значения хранятся в столбцах `children`, `family_status` и `debt`, ведь они понадобятся нам, чтобы ответить на поставленные вопросы.

В столбце `debt` все хорошо: там только нули и единицы. Так и должно быть.

In [8]:
df['debt'].unique()

array([0, 1])

В столбце `family_status` обноружилось одно значение, в котором есть заглавная буква. Это нестрашно, так как копии только со строчными буквами у него нет.

In [9]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

С детьми, к сожалению, не так, как с долгами и семейным положением. В столбце `children` обнаружились отрицательные и аномально большие значения.

In [10]:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Сначала я подумал, что 20 — это выброс. Но оказалось, что таких значений 76, а значит, это, скорее всего, ошибки ввода, которые придется исправлять.

In [11]:
df[df['children']==20].count()

children            76
days_employed       67
dob_years           76
education           76
education_id        76
family_status       76
family_status_id    76
gender              76
income_type         76
debt                76
total_income        67
purpose             76
dtype: int64

#### Вывод

Каждая строка таблицы содержит информацию о клиенте: пол, возраст, образование, семейное положение, количество детей, тип занятости и трудовой стаж, ежемесячный доход, а также цель получения кредита и была ли задолженность по возврату кредита в прошлом.

Количество значений в столбцах различается, значит, в данных есть пропуски. Также есть проблемы со значениями в некоторых столбцах:
1. В `education` значения записаны в разных регистрах;
2. В `days_employed` отрицательными числами записан стаж в днях, а положительными — в часах. Когда исправим это, можно будет заменить тип данных на целочисленный.
3. В `children` — отрицательные значения и опечатки.

Для исследования особенно важны столбцы `family_status`, `children` и `debt`. Столбец `total_income` пригодится, чтобы узнать влиянение дохода на возврат кредита в срок, а `purpose` — влияение цели на возврат.

[К оглавлению](#Оглавление)

## 2. Предобработка данных

### Обработка артефактов

Перед тем как обрабатывать пропуски, исправим артефакты в столбцах `days_employed` и `children`. Это надо сделать до заполнения пропусков, потому что артефакты и выбросы могут испортить обработку.

#### Столбец days_employed

Удалим выбросы — строчки с безработными клиентами.

In [12]:
unemployed_indexes = df[df['income_type']=='безработный'].index
df = df.drop(unemployed_indexes).reset_index(drop=True)

Проверим, что безработных клиентов больше нет в датафрейме.

In [13]:
len(df[df['income_type']=='безработный'])

0

Так и есть! То есть так и нет.

Теперь со стажем в часах остались только пенсионеры. Переведем эти значения в дни. Для этого напишем функцию `hours_to_days`, которая проверяет тип занятости клиента и, если он пенсионер, то возвращает стаж, поделенный на 24. Применим это функцию к датафрейму `df`.

In [14]:
def hours_to_days(row):
    days_employed = row['days_employed']
    income_type = row['income_type']
    if income_type == 'пенсионер':
        days = days_employed / 24
        return days
    return days_employed
    
df['days_employed'] = df.apply(hours_to_days, axis=1)
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21518,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21519,0,14330.725172,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21520,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21521,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


Проверим, что теперь у пенсинеров количество рабочих дней снизилось с 300-400 тысяч, до 14-16 тысяч.

In [15]:
df[df['income_type']=='пенсионер']['days_employed']

4        14177.753002
12                NaN
18       16678.380705
24       14106.331371
25       15147.853723
             ...     
21503    14121.036100
21506    16104.071420
21507    15090.043922
21516    15583.154618
21519    14330.725172
Name: days_employed, Length: 3856, dtype: float64

Получилось! NaN нам не страшен, мы избавимся от него позже.

Теперь заменим отрицательные значения на положительные. Это легко сделать с помощью функции `abs()`.

In [16]:
df['days_employed'] = df['days_employed'].abs()

Проверка:

In [17]:
len(df[df['days_employed'] < 0])

0

Работает!

#### Столбец children

Вспомним, в чем там дело: столбец содержит отрицательные значения и опечатки (20 детей вместо 2). Сначала исправим отрицательные значения тем же методом apply(), который использовали в прошлом шаге. И сразу сделаем проверку.

In [18]:
df['children'] = df['children'].abs()
len(df[df['days_employed'] < 0])

0

Получилось. Теперь заменим все двадцатки на двойки. Для этого применим к столбцу `children` метод `replace()`. Проверим, что супермногодетных семей больше нет.

In [19]:
df['children'] = df['children'].replace(20, 2)
len(df[df['children']==20])

0

#### Вывод

Мы подготовили столбцы `days_employed` и `children` к обработке пропусков. В столбце `days_employed` мы избавились от выбросов, починили размерность стажа у пенсионеров и сделали отрицательне значения положительными. В столбце `children` — исправили опечатки.

### Обработка пропусков

Определим, в каких столбцах есть пропущенные значения и сколько их.

In [20]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Видно, что пропуски есть в столбцах `days_employed` и `total_income`, причем их одинаковое количество. Возможно, они находятся в одних и тех же строчках. Проверим этом:

In [21]:
df[(df['days_employed'].isnull() == True) & (df['total_income'].isnull() == True)].isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Количество пропусков в этом и прошлом выводах совпадает. Это значит, что пропуски стажа и дохода действительно находятся в одинаковых строчках. Посмотрим на них.

In [22]:
df[(df['days_employed'].isnull() == True) & (df['total_income'].isnull() == True)].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Я не вижу связи между пропусками и значениями в других столбцах. Есть три предположения, почему столбцы пустуют:
1. Эти клиенты никогда не работали и у них нет ежемясячного дохода. *(Откуда тогда у них кредиты?..)*
2. Они не заполнили соответсвующие поля в анкете.
3. Прозошел сбой при выгрузке данных.

Строк с пропусками 2174 — это 10% от всех значений, то есть данных слишком много, чтобы избавлять от них. Заполним пропуски.

Пропущенные значения количественные, поэтому заполним их характерными данными. Для этого возьмем медианные значения.

In [23]:
days_employed_median = df['days_employed'].median()
total_income_median = df['total_income'].median()
df['days_employed'] = df['days_employed'].fillna(days_employed_median)
df['total_income'] = df['total_income'].fillna(total_income_median)

Проверим, что пропусков больше нет.

In [24]:
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

#### Вывод

Мы обнаружили пропуски в столбцах `days_employed` и `total_income`. Оказалось, что они находятся в одних и тех же строчках. Это дало нам три потенциальные причины, откуда они взялись:
1. Клиенты с пропусками никогда не работали и у них нет ежемясячного дохода. *(Откуда тогда у них кредиты?..)*
2. Они не заполнили соответсвующие поля в анкете.
3. Прозошел сбой при выгрузке данных.

Пропущенных значений оказалось слишком много, поэтому мы не стали удалять строчки с ними. Вместо этого мы заполнили пропуски медианными значениями столбцов, в которых они находятся.

### Замена типа данных

В столбцах `days_employed` и `total_income` заменим вещественный тип данных на целочисленный. Для этого воспользуемся методом `astype()`.

In [25]:
df['days_employed'] = df['days_employed'].astype('uint16')
df['total_income'] = df['total_income'].astype('uint32')
df['children'] = df['children'].astype('uint8')

Проверим, что теперь в столбцах `days_employed` и `total_income` стоят «инты».

In [26]:
df.dtypes

children             uint8
days_employed       uint16
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income        uint32
purpose             object
dtype: object

### Обработка дубликатов

Перед тем как искать дубликаты, надо обработать столбец `education`, в котором одинаковые значения записаны в разных регистрах. Иначе метод `drop_duplicates()` посчитает такие значения разными. Заменим все заглавные буквы на строчные.

In [27]:
df['education'] = df['education'].str.lower()

Удостоверимся, что теперь все значения записаны строчными буквами.

In [28]:
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Теперь можно удалить повторяющиеся строчки. Для этого воспользуемся методом `drop_duplicates()`, который удаляет все повторяющиеся строчки. При вызове метода вместе с повторяющимися строками удаляются их индексы, поэтому применим метод `reset_index()`, который обновит индексы.

In [29]:
df = df.drop_duplicates().reset_index(drop=True)

Дубликаты могли появиться из-за того, что клиент подал две заявки одновременно, например, из-за того что завис сайт. Или из-за сбоя в банковской системе. Меня удивило, что у всех дупликатов в столбце `days_employed` стояло значение 2194. Мне кажется, эта особенность указывает на сбой в системе, а не на человеческий фактор.

Проверим, что дубликатов больше нет.

In [30]:
df.duplicated().sum()

0

#### Вывод

Мы избавились от дубликатов в датафрейме. Чтобы это сделать, нам понадобилось поменять все заглавные буквы в столбце `education` на строчные. С этим мы тоже справились!

### Лемматизация

В следующем пункте мы будем категоризовать клиентов. Одним из параметров категоризации станет цель кредита. Одна и та же цель кредита может быть записана по-разному, например, «на покупку машины» и «на приобритение машины». Это помешает нашей категоризации, потому что для каждой категории придется составлять словарь строк.

Оптимизировать процесс поможет лемматизация. После нее все слова в целях будут в нормальной (словарной) форме. Поэтому мы сможем относить цель к той или иной категории, смотря на наличие всего одного слова (названия категории), а не большого списка словоформ.

Чтобы провести лемматизацию, нам понадобится лемматайзер `Mystem()` из библиотеки `pymystem3`.

In [31]:
m = Mystem()

Теперь добавим в датафрейм столбец с результатами лемматизации целей кредита. Объединим леммы в одну строку: так будет проще искать значения в ячейке при категоризации. Также удалим символ переноса строки.

In [32]:
def lemmatize_purpose(purpose):
    lem_purpose = ''
    lemmas = m.lemmatize(purpose)
    for lemma in lemmas:
        if lemma != '\n':
            lem_purpose += lemma
    return lem_purpose

df['lem_purpose'] = df['purpose'].apply(lemmatize_purpose)

Посмотрим, получилось ли, и если да, то что именно.

In [33]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lem_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,покупка жилье
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,операция с жилье
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,на проведение свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,покупка жилье для семья


Получилось то, что мы хотели получить!

#### Вывод

Мы добавили в наш датафрейм новую колонку с леммами целей кредита. Для этого мы подключили к нашему проекту библиотеку `pymystem3` и применили ее функцию `lemmatize()` к значениям в столбце `purpose`. 

### Категоризация данных

Категоризовать данные будем по трем параметрам: уровень дохода, цель и количество детей, так как нам надо найти зависимость между этими величинами и возвратом кредита.

#### Доход

Организация экономического сотрудничества и развития [считает](https://www.kommersant.ru/doc/3944361), что средний класс зарабатывает от 75 до 200 процентов от медианной зарплаты. Прислушаемся к этим ребятам и будем считать, что:
* ежемесячный доход до 0,75 медианы — низкий;
* ежемесячный доход от 0,75 до 2 медиан, включая границы — средний;
* ежемесячный доход больше 2 медиан — высокий.

Напишем функцию, которая определяет, какой доход у клиенты. После этого с помощью функции `apply()` применим ее к столбцу `total_income`. Сохраним результат в новый столбец `income_category` и посмотрим на него.

In [34]:
median_total_income = df['total_income'].median()

def set_income_category(month_income):
    if month_income < 0.75 * median_total_income:
        return 'низкий'
    elif 0.75 * median_total_income <= month_income <= 2 * median_total_income:
        return 'средний'
    return 'высокий'

df['income_category'] = df['total_income'].apply(set_income_category)
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lem_purpose,income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье,средний
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль,средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье,средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование,средний
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба,средний
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,покупка жилье,средний
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,операция с жилье,средний
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,средний
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,на проведение свадьба,низкий
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,покупка жилье для семья,средний


#### Цель

Посмотрим на уникальные значения в столбце с леммами.

In [35]:
df['lem_purpose'].unique()

array(['покупка жилье', 'приобретение автомобиль',
       'дополнительный образование', 'сыграть свадьба',
       'операция с жилье', 'образование', 'на проведение свадьба',
       'покупка жилье для семья', 'покупка недвижимость',
       'покупка коммерческий недвижимость', 'покупка жилой недвижимость',
       'строительство собственный недвижимость', 'недвижимость',
       'строительство недвижимость', 'на покупка подержать автомобиль',
       'на покупка свой автомобиль',
       'операция с коммерческий недвижимость',
       'строительство жилой недвижимость', 'жилье',
       'операция со свой недвижимость', 'автомобиль',
       'заниматься образование', 'сделка с подержанный автомобиль',
       'получение образование', 'свадьба',
       'получение дополнительный образование', 'покупка свой жилье',
       'операция с недвижимость', 'получение высокий образование',
       'свой автомобиль', 'сделка с автомобиль', 'профильный образование',
       'высокий образование', 'покупка жилье 

Из них можно выделить четыре большие категории:
1. недвижимость — отнесем к ней все значения, в которых встречается строка `'недвижимость'` или `'жил'`;
2. автомобиль
3. образование
4. свадьба

Напишем функцию, которая ищет в строчке с леммами ключевые слова категорий. Применим ее к столбцe `lem_purpose`. Сохраним результат в новый столбец `purpose_category` и посмотрим на него.

In [36]:
def set_purpose_category(lem_purpose):
    if ('недвижимость' in lem_purpose) | ('жил' in lem_purpose):
        return 'недвижимость'
    elif 'автомобиль' in lem_purpose:
        return 'автомобиль'
    elif 'образование' in lem_purpose:
        return 'образование'
    elif 'свадьба' in lem_purpose:
        return 'свадьба'

df['purpose_category'] = df['lem_purpose'].apply(set_purpose_category)
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lem_purpose,income_category,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье,средний,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль,средний,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье,средний,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование,средний,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба,средний,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,покупка жилье,средний,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,операция с жилье,средний,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,средний,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,на проведение свадьба,низкий,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,покупка жилье для семья,средний,недвижимость


#### Дети

Посмотрим, как наши клиенты распределяются по количеству детей.

In [37]:
df['children'].value_counts()

0    14090
1     4854
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Заметим, что в сумме клиенты с 3, 4 и 5 детьми составляют менее двух процентов от всех клиентов. Поэтому для категоризации возьмем следующие группы:
* нет детей,
* 1 ребенок,
* 2 и более детей.

Напишем функцию, которая присваивает клиенту категорию взависимости от количества детей. Применим ее к столбцу `children`. Сохраним результат в новый столбец `children_category` и посмотрим на него.

In [38]:
def set_children_category(children):
    if children == 0:
        return 'нет детей'
    elif children == 1:
        return '1 ребенок'
    return '2 и более детей'

df['children_category'] = df['children'].apply(set_children_category)
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lem_purpose,income_category,purpose_category,children_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье,средний,недвижимость,1 ребенок
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль,средний,автомобиль,1 ребенок
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье,средний,недвижимость,нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование,средний,образование,2 и более детей
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба,средний,свадьба,нет детей
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,покупка жилье,средний,недвижимость,нет детей
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,операция с жилье,средний,недвижимость,нет детей
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,средний,образование,нет детей
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,на проведение свадьба,низкий,свадьба,2 и более детей
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,покупка жилье для семья,средний,недвижимость,нет детей


#### Вывод

Мы категоризовали данные по трем параметрам: ежемесячный доход клиента, цель кредита и количество детей:

- Чтобы определить категории для первого параметра, мы узнали, что считается низким, средним и высоким доходом по версии Организации экономического сотрудничества и развития.

- Категории целей мы выделили, изучив леммы целей. 

- Чтобы определиться с категориями детей, мы посмотрели, как распределены наши клиенты по количеству детей.

В результате в таблице появились три новых столбца: `income_category` с категорией дохода клиента, `purpose_category` с категорией цели кредита и `children_category` с категорий количества детей.

[К оглавлению](#Оглавление)

## 3. Ответы на вопросы

У кредитного отдела есть несколько вопросов по данным. Он просит нас помочь ему с ответами.

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Чтобы ответить на этот вопрос составим сводную таблицу. В качестве индексов будем использовать категорию количества детей. В столбцах укажем общее число клиентов и сколько из них должники. Для этого применим к значениям функции `count()` и `sum()`. Сумма подходит для подсчета должников, потому что у них в стобце `debt` стоят единицы.

In [39]:
children_pivot_table = df.pivot_table(index='children_category', values='debt', aggfunc=['count', 'sum'])

Посмотрим на результат.

In [40]:
children_pivot_table

,count,sum
,debt,debt
children_category,,
1 ребенок,4854,444
2 и более детей,2508,233
нет детей,14090,1063


Дадим столбцам более понятные названия.

In [41]:
children_pivot_table.columns = list(map("_".join, children_pivot_table.columns))
children_pivot_table.rename(columns={'count_debt': 'clients', 'sum_debt': 'debtors'}, inplace=True)

Добавим к сводной таблице еще один столбец, в котором будет указано, какой процент от клиентов составляют должники. Переведем это значение в проценты и округлим до двух знаков после запятой.

In [42]:
children_pivot_table['percentage'] = children_pivot_table['debtors'] / children_pivot_table['clients']
children_pivot_table['percentage'] = round(children_pivot_table['percentage'] * 100, 2)
children_pivot_table['percentage'] = children_pivot_table['percentage'].astype('str') + '%'

Отсортируем полученную таблицу по убыванию по столбцу 'percentage', чтобы было проще сделать вывод. Посмотрим на результат.

In [43]:
children_pivot_table.sort_values(by='percentage', ascending=False)

,clients,debtors,percentage
children_category,,,
2 и более детей,2508,233,9.29%
1 ребенок,4854,444,9.15%
нет детей,14090,1063,7.54%


#### Вывод

Данные показывают, что чем больше у клиента детей, тем чаще он задерживает выплату кредита. Скорее всего, это связано с тем, что на детей приходится много дополнительных трат, которые наверняка приоритетнее выплаты долга.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Воспользуемся последовательностью действий из прошлого ответа как шаблоном и на его основе составим новую таблицу. В этот раз в качестве индексов будет использоваться семейное положение клиентов.

In [44]:
family_status_pivot_table = df.pivot_table(index='family_status', values='debt', aggfunc=['count', 'sum'])
family_status_pivot_table.columns = list(map("_".join, family_status_pivot_table.columns))
family_status_pivot_table.rename(columns={'count_debt': 'clients', 'sum_debt': 'debtors'}, inplace=True)
family_status_pivot_table['percentage'] = family_status_pivot_table['debtors'] / family_status_pivot_table['clients']
family_status_pivot_table['percentage'] = round(family_status_pivot_table['percentage'] * 100, 2)
family_status_pivot_table['percentage'] = family_status_pivot_table['percentage'].astype('str') + '%'
family_status_pivot_table.sort_values(by='percentage', ascending=False)

,clients,debtors,percentage
family_status,,,
Не женат / не замужем,2810,274,9.75%
гражданский брак,4150,388,9.35%
женат / замужем,12338,930,7.54%
в разводе,1195,85,7.11%
вдовец / вдова,959,63,6.57%


#### Вывод

Приведенная выше таблица показывает, что связь между семейным положеним клиента и своевременным возвратом кредита есть. Клиенты, которые не состояли в официальном браке, задерживают выплаты чаще.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Снова применим шаблон из первого ответа. За индексы возьмем уровни дохода, которые мы дали клиентам при категоризации данных.

In [45]:
income_category_pivot_table = df.pivot_table(index='income_category', values='debt', aggfunc=['count', 'sum'])
income_category_pivot_table.columns = list(map("_".join, income_category_pivot_table.columns))
income_category_pivot_table.rename(columns={'count_debt': 'clients', 'sum_debt': 'debtors'}, inplace=True)
income_category_pivot_table['percentage'] = income_category_pivot_table['debtors'] / income_category_pivot_table['clients']
income_category_pivot_table['percentage'] = round(income_category_pivot_table['percentage'] * 100, 2)
income_category_pivot_table['percentage'] = income_category_pivot_table['percentage'].astype('str') + '%'
income_category_pivot_table.sort_values(by='percentage', ascending=False)

,clients,debtors,percentage
income_category,,,
средний,14297,1183,8.27%
низкий,5500,437,7.95%
высокий,1655,120,7.25%


#### Вывод

Я не вижу зависимости между доходом и возвратом кредита в срок. Люди с низким и средним доходом не возвращают кредиты почти с одинаковой частотой. Люди с высоким доходом не возвращают кредиты реже, но этот результат может быть связан с тем, что это самая маленькая группа клиентов.

### Как разные цели кредита влияют на его возврат в срок?

Снова шаблон, но теперь в качестве индексов — категории целей кредита.

In [46]:
purpose_category_pivot_table = df.pivot_table(index='purpose_category', values='debt', aggfunc=['count', 'sum'])
purpose_category_pivot_table.columns = list(map("_".join, purpose_category_pivot_table.columns))
purpose_category_pivot_table.rename(columns={'count_debt': 'clients', 'sum_debt': 'debtors'}, inplace=True)
purpose_category_pivot_table['percentage'] = purpose_category_pivot_table['debtors'] / purpose_category_pivot_table['clients']
purpose_category_pivot_table['percentage'] = round(purpose_category_pivot_table['percentage'] * 100, 2)
purpose_category_pivot_table['percentage'] = purpose_category_pivot_table['percentage'].astype('str') + '%'
purpose_category_pivot_table.sort_values(by='percentage', ascending=False)

,clients,debtors,percentage
purpose_category,,,
автомобиль,4306,403,9.36%
образование,4013,370,9.22%
свадьба,2324,186,8.0%
недвижимость,10809,781,7.23%


#### Вывод

Здесь наблюдаются зависимость важности цели кредита и его возврата в срок. Купить недвижимость или сыграть свадьбу — ответственный шаг в жизни человека. Купить или отремонтировать автомобиль — тоже, но все же это другой уровень решительности.

[К оглавлению](#Оглавление)

## 4. Общий вывод

В начале исследования перед нами стояли два вопроса:
1. Влияет ли семейное положение клиента на возврат кредита в срок? 
2. Влияет количество детей клиента на возврат кредита в срок?

Иследование показало, что оба показателя влияют на своевременный возврат кредита, но в разной степени.

Возврат кредит в срок зависит от семейного положения сильнее, чем от количества детей. Вот как своевременная выплата долга зависит от семейного положения клиента:

In [47]:
family_status_pivot_table.sort_values(by='percentage', ascending=False)

,clients,debtors,percentage
family_status,,,
Не женат / не замужем,2810,274,9.75%
гражданский брак,4150,388,9.35%
женат / замужем,12338,930,7.54%
в разводе,1195,85,7.11%
вдовец / вдова,959,63,6.57%


А вот как просрочка зависит от количества детей:

In [48]:
children_pivot_table.sort_values(by='percentage', ascending=False)

,clients,debtors,percentage
children_category,,,
2 и более детей,2508,233,9.29%
1 ребенок,4854,444,9.15%
нет детей,14090,1063,7.54%


Коротко эту зависимость можно описать так: чем больше у клиентов детей, тем чаще они задерживают выплату кредита.

Также предоставленные данные позволили нам исследовать зависимость возврата кредита в срок от цели кредита и уровня дохода клиента. Оказалось, что чем ответсвеннее и масштабнее цели клиента, тем реже они задерживают выплату.

In [49]:
purpose_category_pivot_table.sort_values(by='percentage', ascending=False)

,clients,debtors,percentage
purpose_category,,,
автомобиль,4306,403,9.36%
образование,4013,370,9.22%
свадьба,2324,186,8.0%
недвижимость,10809,781,7.23%


А вот зависимость возврата кредита в срок от уровня дохода клиента установить не удалось.

In [50]:
income_category_pivot_table.sort_values(by='percentage', ascending=False)

,clients,debtors,percentage
income_category,,,
средний,14297,1183,8.27%
низкий,5500,437,7.95%
высокий,1655,120,7.25%


Небольшое «отставание» клиентов с высоким доходом объясняется малым объемом данных о них.

[К оглавлению](#Оглавление)